## Prerequisites

In [ ]:
import sys
sys.path.append("..")

from collections import defaultdict
from chaininglib.search.metadata import get_available_metadata
from chaininglib.utils.dfops import df_filter, property_freq
from chaininglib.ui.dfui import display_df
from chaininglib.search.CorpusQuery import *
from chaininglib.process.corpus import get_frequency_list
from chaininglib.utils.dfops import join_df


# TODO: query all lemmas (not just b), maybe only NOU to keep it lighter
corpus_name="zeebrieven"
query=r'[pos="NOU"]'
fields = get_available_metadata(corpus_name)
c = create_corpus(corpus_name).pattern(query).extra_fields_doc(fields["document"]).search()
df_corpus = c.kwic()



df = {}
df["17th"]= df_corpus[df_corpus["witnessYear_from"] < 1700] 
df["18th"] = df_corpus[df_corpus["witnessYear_from"] > 1700] 

# Then, filter Zeebrieven per sociolinguistic variable (social class/occupation)
# Take only extremes: high and low. Remove middle-low and middle-high
freq_list = defaultdict(lambda: dict())
for era, df_era in df.items():
    for soc_class in ["low", "high"]:
        cond = df_filter(df_era["afz_klasse"], pattern=soc_class)
        df_era_class = df_era[ cond ]
        freq_list[era][soc_class] = get_frequency_list(df_era_class)

from chaininglib.utils.dfops import get_relfreq_diff
SHOW=100
df_relfreq_diff = {}
df_relfreq_diff["class_diff_17th"] = get_relfreq_diff(freq_list["17th"]["low"], freq_list["17th"]["high"], index="lemmas", label1="lower class", label2="upper class" )
df_relfreq_diff["class_diff_18th"] = get_relfreq_diff(freq_list["18th"]["low"], freq_list["18th"]["high"], index="lemmas", label1="lower class", label2="upper class")
df_relfreq_diff["low_class_evolution"] = get_relfreq_diff(freq_list["17th"]["low"], freq_list["18th"]["low"], index="lemmas", label1="17th century", label2="18th century")
df_relfreq_diff["high_class_evolution"] = get_relfreq_diff(freq_list["17th"]["high"], freq_list["18th"]["high"], index="lemmas",label1="17th century", label2="18th century" )

display_df(df_relfreq_diff["class_diff_17th"].sort_values(by="relfreq_diff", ascending=False).head(SHOW), labels="Salient lower class words 17th century")
display_df(df_relfreq_diff["class_diff_17th"].sort_values(by="relfreq_diff", ascending=True).head(SHOW), labels="Salient upper class words 17th century")

display_df(df_relfreq_diff["class_diff_18th"].sort_values(by="relfreq_diff", ascending=False).head(SHOW), labels="Salient lower class words 18th century")
display_df(df_relfreq_diff["class_diff_18th"].sort_values(by="relfreq_diff", ascending=True).head(SHOW), labels="Salient upper class words 18th century")


display_df(df_relfreq_diff["low_class_evolution"].sort_values(by="relfreq_diff", ascending=False).head(SHOW), labels="Specific 17th century words lower class")
display_df(df_relfreq_diff["low_class_evolution"].sort_values(by="relfreq_diff", ascending=True).head(SHOW), labels="Specific 18th century words lower class")


display_df(df_relfreq_diff["high_class_evolution"].sort_values(by="relfreq_diff", ascending=False).head(SHOW), labels="Specific 17th century words upper class")
display_df(df_relfreq_diff["high_class_evolution"].sort_values(by="relfreq_diff", ascending=True).head(SHOW), labels="Specific 18th century words upper class")




## Task 1: compare social class and period

 * Load data and get frequency

 * Perform comparisons